# DAG Sample

This pipeline implements a [directed acyclic graph (DAG)](https://en.wikipedia.org/wiki/Directed_acyclic_graph) of stage functions.
The pipeline has six stages, `A`, `B`, `C`, `D`, `E`, and `F`.
For the purposes of the demo, each stage sleeps for tens of milliseonds and then completes.
Here is the structure of the DAG:

![](../../documentation/dag.png)

This graph represents the following stage function dependencies:
* `B` and `C` can't run until `A` completes.
* `D` depends on `B` and `C`
* `F` depends on `D` and `E`

We'll start by importing the pipeline:

In [1]:
from dag import dag_pipeline_spec

The DAG pipeline doesn't actually read its test cases, but we need to define one empty case so that Gotaglio's `run()` method will invoke the pipeline.

In [2]:
cases = [
  {
    "uuid": "6919a246-945f-4d93-b39d-d798244dd08e",
  },
]

Instantiate the Gotaglio object that runs the pipeline.

In [3]:
from gotaglio.gotag import Gotaglio

gt = Gotaglio([dag_pipeline_spec])

Run the pipeline. The summarize() method generates a table that shows the execution of the stages over time.
Each row corresponds to a point in time where a stage function started or stopped. The green highlight
indicates when stages were running.

In [4]:
result = gt.run(
  "dag",
  cases,
)

             Timeline for case              
    6919a246-945f-4d93-b39d-d798244dd08e    
┏━━━━━━┳━━━━━┳━━━━━┳━━━━━┳━━━━━┳━━━━━┳━━━━━┓
┃ step ┃  A  ┃  B  ┃  C  ┃  D  ┃  E  ┃  F  ┃
┡━━━━━━╇━━━━━╇━━━━━╇━━━━━╇━━━━━╇━━━━━╇━━━━━┩
│    1 │  x  │     │     │     │     │     │
│    2 │  x  │     │     │     │  x  │     │
│    3 │  x  │     │     │     │  x  │     │
│    4 │     │     │     │     │  x  │     │
│    5 │     │  x  │     │     │     │     │
│    6 │     │  x  │  x  │     │     │     │
│    7 │     │  x  │  x  │     │     │     │
│    8 │     │     │  x  │     │     │     │
│    9 │     │     │     │  x  │     │     │
│   10 │     │     │     │  x  │     │     │
│   11 │     │     │     │     │     │  x  │
│   12 │     │     │     │     │     │  x  │
└──────┴─────┴─────┴─────┴─────┴─────┴─────┘

We can rerun this example with a different model, this time the `flakey` mock, which alternately returns the correct answer, returns "hello world", and raises an exception. The mock models are good for debugging new pipelines locally, before connecting to a real model.

In [5]:
result2 = gt.rerun(
  result,
  {
    "infer.model.name": "flakey"
  },
  save=True
)
gt.format(result2)

             Timeline for case              
    6919a246-945f-4d93-b39d-d798244dd08e    
┏━━━━━━┳━━━━━┳━━━━━┳━━━━━┳━━━━━┳━━━━━┳━━━━━┓
┃ step ┃  A  ┃  B  ┃  C  ┃  D  ┃  E  ┃  F  ┃
┡━━━━━━╇━━━━━╇━━━━━╇━━━━━╇━━━━━╇━━━━━╇━━━━━┩
│    1 │  x  │     │     │     │     │     │
│    2 │  x  │     │     │     │  x  │     │
│    3 │  x  │     │     │     │  x  │     │
│    4 │     │     │     │     │  x  │     │
│    5 │     │  x  │     │     │     │     │
│    6 │     │  x  │  x  │     │     │     │
│    7 │     │  x  │  x  │     │     │     │
│    8 │     │     │  x  │     │     │     │
│    9 │     │     │     │  x  │     │     │
│   10 │     │     │     │  x  │     │     │
│   11 │     │     │     │     │     │  x  │
│   12 │     │     │     │     │     │  x  │
└──────┴─────┴─────┴─────┴─────┴─────┴─────┘


Results written to logs\f3b8f357-a74e-47f5-a037-c65af5617a75.json


             Timeline for case              
    6919a246-945f-4d93-b39d-d798244dd08e    
┏━━━━━━┳━━━━━┳━━━━━┳━━━━━┳━━━━━┳━━━━━┳━━━━━┓
┃ step ┃  A  ┃  B  ┃  C  ┃  D  ┃  E  ┃  F  ┃
┡━━━━━━╇━━━━━╇━━━━━╇━━━━━╇━━━━━╇━━━━━╇━━━━━┩
│    1 │  x  │     │     │     │     │     │
│    2 │  x  │     │     │     │  x  │     │
│    3 │  x  │     │     │     │  x  │     │
│    4 │     │     │     │     │  x  │     │
│    5 │     │  x  │     │     │     │     │
│    6 │     │  x  │  x  │     │     │     │
│    7 │     │  x  │  x  │     │     │     │
│    8 │     │     │  x  │     │     │     │
│    9 │     │     │     │  x  │     │     │
│   10 │     │     │     │  x  │     │     │
│   11 │     │     │     │     │     │  x  │
│   12 │     │     │     │     │     │  x  │
└──────┴─────┴─────┴─────┴─────┴─────┴─────┘

Note that we can use the term `"latest"` to refer to the most recently generated runlog.
We could also pass the first few characters of a run's UUID or we could pass the result
object (e.g. `result` or `result2`) directly.

In [6]:
gt.summarize("latest")

             Timeline for case              
    6919a246-945f-4d93-b39d-d798244dd08e    
┏━━━━━━┳━━━━━┳━━━━━┳━━━━━┳━━━━━┳━━━━━┳━━━━━┓
┃ step ┃  A  ┃  B  ┃  C  ┃  D  ┃  E  ┃  F  ┃
┡━━━━━━╇━━━━━╇━━━━━╇━━━━━╇━━━━━╇━━━━━╇━━━━━┩
│    1 │  x  │     │     │     │     │     │
│    2 │  x  │     │     │     │  x  │     │
│    3 │  x  │     │     │     │  x  │     │
│    4 │     │     │     │     │  x  │     │
│    5 │     │  x  │     │     │     │     │
│    6 │     │  x  │  x  │     │     │     │
│    7 │     │  x  │  x  │     │     │     │
│    8 │     │     │  x  │     │     │     │
│    9 │     │     │     │  x  │     │     │
│   10 │     │     │     │  x  │     │     │
│   11 │     │     │     │     │     │  x  │
│   12 │     │     │     │     │     │  x  │
└──────┴─────┴─────┴─────┴─────┴─────┴─────┘